In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]= "2,3"

In [2]:
import torch
model = torch.hub.load("pytorch/vision", "vit_b_16")

import torchvision
from Models.transformer import VisionTransformer as vit
import Models.Conv as conv

from DataLoader import CIFAR100
from tqdm import tqdm
import torch.optim as optim
import torch.nn as nn

import utils

import numpy as np
import torch.backends.cudnn as cudnn
import random

Using cache found in /root/.cache/torch/hub/pytorch_vision_main
/root/.cache/torch/hub/pytorch_vision_main/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")
Using cache found in /root/.cache/torch/hub/pytorch_vision_main
/root/.cache/torch/hub/pytorch_vision_main/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Using cache found in /root/.cache/torch/hub/pytorch_vision_main
/root/.cache/torch/hub/pytorch_vision_main/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrain

In [3]:
train_loader, test_loader = CIFAR100.get_data(58*3)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
teacher = vit(class_num = 100, pretrained = True)
teacher.load_state_dict(torch.load("saved_models/vit_b_16_m_0_9_std_0_4_acc_89_01.pth").module.state_dict())


<All keys matched successfully>

In [5]:
device = "cuda"

teacher = teacher.to(device)
teacher = torch.nn.DataParallel(teacher, device_ids=[0, 1])


In [6]:
criterion_onlylabel = lambda a,b : mse(a*b, b)
criterion_CE = nn.CrossEntropyLoss()
mse = nn.MSELoss()
softmax = torch.nn.Softmax(dim = 1)
criterion_KLD = torch.nn.KLDivLoss(reduction="batchmean")
criterion_response = lambda a,b : criterion_KLD(torch.log_softmax(a, dim=1),torch.softmax(b, dim=1))


In [7]:
T_optimizer = optim.SGD(teacher.parameters(), lr=0.01, momentum=0.9)
CE_loss = nn.CrossEntropyLoss()
T_scheduler = torch.optim.lr_scheduler.MultiStepLR(T_optimizer, milestones=[1,2,3,4,5,6,7], gamma=0.1)

In [8]:
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
np.random.seed(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)
best_acc = 0.0
stack = 0

accs_train = []
accs_test = []

try:
    encoder_length = len(teacher.encoder.layers)
except:
    encoder_length = len(teacher.module.encoder.layers)


T_correct = 0
all_data = 0

for i, (img, label) in enumerate(tqdm(train_loader)):
    teacher.train()
    input_data = img.to(device)
    label = label.to(device)


    all_data += len(input_data)
    input_lrp = utils.get_LRP_img(input_data, label, teacher, criterion_CE, T_optimizer, mean=1.5, std = 0.1, mult = 0.4).cuda()

    T_optimizer.zero_grad()

    layer = random.randint(0,  2+encoder_length)

    output_T, fk_lrp = teacher(input_lrp,layer)


    T_correct += sum(label == torch.argmax(output_T, dim=1))
    
    if i % 10 == 0:
        print(f"Teacher acc: {T_correct / all_data}")
        
    
# print("distill loss : ", sum(loss_distill) / len(loss_distill))
# print("general loss : ", sum(loss_CE) / len(loss_CE))
# print("response loss : ", sum(loss_response) / len(loss_response))



  0%|          | 1/288 [00:07<34:51,  7.29s/it]

Teacher acc: 0.9712643623352051


  4%|▍         | 11/288 [00:14<03:25,  1.35it/s]

Teacher acc: 0.9801462888717651


  7%|▋         | 21/288 [00:20<03:07,  1.42it/s]

...

 94%|█████████▍| 271/288 [03:03<00:11,  1.45it/s]

Teacher acc: 0.9731941819190979


 98%|█████████▊| 281/288 [03:10<00:04,  1.46it/s]

Teacher acc: 0.9731664657592773


100%|██████████| 288/288 [03:14<00:00,  1.48it/s]


In [9]:

print(f"Teacher acc: {T_correct / all_data}")


Teacher acc: 0.973099946975708


In [10]:
input_lrp.shape

torch.Size([62, 3, 224, 224])

In [11]:
input_lrp.std(dim=(2,3)).shape

torch.Size([62, 3])